<a href="https://colab.research.google.com/github/DarshanSajeev/COMP2611/blob/main/Copy_of_SearchExercise5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0   174k      0 --:--:-- --:--:-- --:--:--  174k
Loading bbSearch Version 2.1 (at 13:57, Thu 15 Sep)
Last module source code edit 9am Thursday 24th Feb 2022


In [ ]:
class Robot:
    def __init__(self, location, carried_items, strength):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength

    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.carried_items])

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location,
                      self.carried_items,
                      self.strength ) )

class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [ ]:
class State:
    def __init__( self, robot, doors, room_contents ):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents

    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )


In [ ]:
ROOM_CONTENTS = {
    'workshop'     : {'rusty key'},
    'store room'   : {'bucket', 'suitcase'},
    'tool cupboard' : {'sledge hammer', 'anvil', 'saw', 'screwdriver'},
}

ITEM_WEIGHT = {
      'rusty key' : 0,
         'bucket' : 2,
       'suitcase' : 4,
    'screwdriver' : 1,
  'sledge hammer' : 5,
          'anvil' : 12,
            'saw' : 2,
}

DOORS = [
    Door('workshop', 'store room' ),
    Door( 'store room', 'tool cupboard', doorkey='rusty key', locked=False )
]


In [ ]:
from copy import deepcopy

class RobotWorker( SearchProblem ):

    def __init__( self, state, goal_item_locations ):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations

    def possible_actions( self, state ):

        robot_location = state.robot.location
        strength       = state.robot.strength
        weight_carried = state.robot.weight_carried()

        actions = []
        # Can put down any carried item
        for i in state.robot.carried_items:
            actions.append( ("put down", i) )

        # Can pick up any item in room if strong enough
        for i in state.room_contents[robot_location]:
            if strength >= weight_carried + ITEM_WEIGHT[i]:
                actions.append( ("pick up", i))

        # If there is an unlocked door between robot location and
        # another location can move to that location
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append( ("move to", door.other_loc[robot_location]) )

        # Now the actions list should contain all possible actions
        return actions

    def successor( self, state, action):
        next_state = deepcopy(state)
        act, target = action
        if act== "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)

        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == "move to":
            next_state.robot.location = target

        return next_state

    def goal_test(self, state):
        #print(state.room_contents)
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
        return True

    def display_state(self,state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)

In [ ]:
rob = Robot('store room', [], 15 )

state = State(rob, DOORS, ROOM_CONTENTS)

goal_item_locations =  {"store room":{"sledge hammer", "screwdriver", "anvil"}}

RW_PROBLEM_1 = RobotWorker( state, goal_item_locations )

In [ ]:
poss_acts = RW_PROBLEM_1.possible_actions( RW_PROBLEM_1.initial_state )
poss_acts

[('pick up', 'suitcase'),
 ('pick up', 'bucket'),
 ('move to', 'workshop'),
 ('move to', 'tool cupboard')]

In [ ]:
for act in poss_acts:
    print("Action", act, "leads to the following state:")
    next_state = RW_PROBLEM_1.successor( RW_PROBLEM_1.initial_state, act )
    RW_PROBLEM_1.display_state(next_state)
    print()

Action ('pick up', 'suitcase') leads to the following state:
Robot location: store room
Robot carrying: ['suitcase']
Room contents: {'workshop': {'rusty key'}, 'store room': {'bucket'}, 'tool cupboard': {'sledge hammer', 'saw', 'anvil', 'screwdriver'}}

Action ('pick up', 'bucket') leads to the following state:
Robot location: store room
Robot carrying: ['bucket']
Room contents: {'workshop': {'rusty key'}, 'store room': {'suitcase'}, 'tool cupboard': {'sledge hammer', 'saw', 'anvil', 'screwdriver'}}

Action ('move to', 'workshop') leads to the following state:
Robot location: workshop
Robot carrying: []
Room contents: {'workshop': {'rusty key'}, 'store room': {'suitcase', 'bucket'}, 'tool cupboard': {'sledge hammer', 'saw', 'anvil', 'screwdriver'}}

Action ('move to', 'tool cupboard') leads to the following state:
Robot location: tool cupboard
Robot carrying: []
Room contents: {'workshop': {'rusty key'}, 'store room': {'suitcase', 'bucket'}, 'tool cupboard': {'sledge hammer', 'saw', 'a

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 100000, loop_check=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
...................
:-)) *SUCCESS* ((-:

Path length = 10
Goal state is:
Robot location: store room
Robot carrying: []
Room contents: {'workshop': {'rusty key'}, 'store room': {'screwdriver', 'anvil', 'bucket', 'suitcase', 'sledge hammer'}, 'tool cupboard': {'saw'}}
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('pick up', 'sledge hammer')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'sledge hammer')
    ('put down', 'screwdriver')
    ('move to', 'tool cupboard')
    ('pick up', 'anvil')
    ('move to', 'store room')
    ('put down', 'anvil')


SEARCH SPACE STATS:
Total nodes generated          =   101789  (includes start)


'GOAL_STATE_FOUND'